In [12]:
import geopandas as gpd
df1 = (
    gpd.read_file(r"C:\Users\smcdonald\Documents\Data\LULCC_2024ed\AccuracyAssessment\clean_points\fuzzy_TA\DC_lcc.dbf")
    .drop('geometry', axis=1)
    .set_index("VALUE")
)
df2 = (
    gpd.read_file(r"C:\Users\smcdonald\Documents\Data\LULCC_2024ed\AccuracyAssessment\clean_points\fuzzy_TA\DCextent_lcc.dbf")
    .drop('geometry', axis=1)
    .set_index("VALUE")
)

In [ ]:
df1.loc[:, 'lcc_dc'] = df1.idxmax(axis=1).str.split('_', n=1, expand=True)[1]
df2.loc[:, 'lcc_dcext'] = df2.idxmax(axis=1).str.split('_', n=1, expand=True)[1]

df1 = df1[['lcc_dc']]
df2 = df2[['lcc_dcext']]

df = df1.merge(df2, left_index=True, right_index=True, how='outer')

df

In [21]:
pts = (
    gpd.read_file(r"C:\Users\smcdonald\Documents\Data\LULCC_2024ed\AccuracyAssessment\clean_points\lcc_aa_points_cleaned.gpkg", layer='AA_clean')
    .query("state == 'DC'")
    .filter(items=['uid', 'type', 'strata_cls', 'StrataMatch', 'geometry'])
    .merge(df, left_on='uid', right_index=True, how='left')
    .fillna(0)
)

pts.loc[:, 'lcc_dc'] = pts.lcc_dc.astype(int)
pts.loc[:, 'lcc_dcext'] = pts.lcc_dcext.astype(int)

In [ ]:
dc = len(pts.query("strata_cls == lcc_dc and strata_cls != 0"))
dce = len(pts.query("strata_cls == lcc_dcext and strata_cls != 0"))
comp = len(pts.query("lcc_dc == lcc_dcext"))
comp2 = len(pts.query("lcc_dc != lcc_dcext"))


# print(f"DC = DCE: {comp}")
# print(f"DC != DCE: {comp2}")
# print(f"DC = strata: {dc}")
# print(f"DCE = strata: {dce}")

pts.query("lcc_dc != lcc_dcext")


In [40]:
import pandas as pd
import geopandas as gpd
import numpy as np
from openpyxl import load_workbook

# merge distance to T/F values to plot
df = pd.read_csv(r"C:\Users\smcdonald\Documents\Data\LULCC_2024ed\AccuracyAssessment\clean_points\distance\DC_distance.csv")

# read in points
gdf = gpd.read_file(r"C:\Users\smcdonald\Documents\Data\LULCC_2024ed\AccuracyAssessment\clean_points\lcc_aa_points_cleaned.gpkg", layer="AA_clean")

# compute T/F of valid
gdf.loc[:, 'correct'] = False
gdf.loc[gdf['GrndTruth'] == gdf['origMap'], "correct"] = True

# merge to df
df = df.merge(gdf[['uid', 'correct']], on='uid', how='left')
gdf = None

# get fuzzy accuracy value
df2 = (
    pd.read_excel(r"C:\Users\smcdonald\Documents\Data\LULCC_2024ed\AccuracyAssessment\clean_points\fuzzy_TA\summary_tables\DC_fuzzy3x3_tables.xlsx", sheet_name="all_data")
    .query("type == 'change'")
    .query("GT_ch_3x3.notna()")
    .filter(items=['uid', 'GT_ch_3x3', 'Map_static', 'GT_static', 'GT_st_3x3'])
)
df2.loc[:, "GT_ch_3x3"] = df2.GT_ch_3x3.astype(bool)
df2.loc[:, 'st_correct'] = False
df2.loc[df2['Map_static'] == df2['GT_static'], "st_correct"] = True

# merge fuzzy accuracy to table
df = df.merge(df2[['uid', 'GT_ch_3x3']], on='uid')
df.loc[:, 'T'] = 0
df.loc[df['correct'], 'T'] = 1
df.loc[:, 'F'] = 0
df.loc[~df['correct'], 'F'] = 1
df.loc[:, 'T_3x3'] = 0
df.loc[df['GT_ch_3x3'], 'T_3x3'] = 1
df.loc[:, 'F_3x3'] = 0
df.loc[~df['GT_ch_3x3'], 'F_3x3'] = 1

# write data
out_path = r"C:\Users\smcdonald\Documents\Data\LULCC_2024ed\AccuracyAssessment\clean_points\distance\distance_calc.xlsx"
# df.to_excel(out_path, sheet_name="distanceXcorrect", index=False)

# assign bins
dst_fields = ['dist_cntr_lcc', 'dist_cntr_lccDis']
for dist_fld in dst_fields:
    mx = int(np.amax(df[dist_fld])+1)
    for i in range(mx+1):
        df.loc[(df[dist_fld]<=i) & (df[dist_fld]>i-1), 'bin'] = i


    df1 = df[['bin', 'T', 'F', 'T_3x3', 'F_3x3']].groupby(['bin']).sum().reset_index()

    # book = load_workbook(out_path)
    # writer = pd.ExcelWriter(out_path, engine = 'openpyxl')
    # writer.book = book
    # df1.to_excel(writer, sheet_name=dist_fld, index=False)
    # writer.close()

# evaluate static accuracy by distance in change patch
st_df = df[['uid', 'dist_orig_lcc', 'dist_orig_lccDis', 'dist_cntr_lcc', 'dist_cntr_lccDis']].merge(df2[['uid', 'st_correct', 'GT_st_3x3']], on='uid')
st_df.loc[:, 'T'] = 0
st_df.loc[st_df['st_correct'], 'T'] = 1
st_df.loc[:, 'F'] = 0
st_df.loc[~st_df['st_correct'], 'F'] = 1
st_df.loc[:, 'T_3x3'] = 0
st_df.loc[st_df['GT_st_3x3'], 'T_3x3'] = 1
st_df.loc[:, 'F_3x3'] = 0
st_df.loc[~st_df['GT_st_3x3'], 'F_3x3'] = 1

# assign bins
dst_fields = ['dist_cntr_lcc', 'dist_cntr_lccDis']
for dist_fld in dst_fields:
    mx = int(np.amax(st_df[dist_fld])+1)
    for i in range(mx+1):
        st_df.loc[(st_df[dist_fld]<=i) & (st_df[dist_fld]>i-1), 'bin'] = i


    df1 = st_df[['bin', 'T', 'F', 'T_3x3', 'F_3x3']].groupby(['bin']).sum().reset_index()

    book = load_workbook(out_path)
    writer = pd.ExcelWriter(out_path, engine = 'openpyxl')
    writer.book = book
    df1.to_excel(writer, sheet_name=f"st_{dist_fld}", index=False)
    writer.close()


In [42]:
import pandas as pd
import geopandas as gpd
import numpy as np
from openpyxl import load_workbook

# distance to change -- merge with other dist data
df = (
    gpd.read_file(r"C:\Users\smcdonald\Documents\Data\LULCC_2024ed\AccuracyAssessment\LCC_AA\LCC_AA.gdb", layer="ZonalSt_main1")
    .filter(items=['uid', 'MAX'], axis=1)
    .rename(columns={'MAX':'dist_to_change'})
)

# get fuzzy accuracy value
df2 = (
    pd.read_excel(r"C:\Users\smcdonald\Documents\Data\LULCC_2024ed\AccuracyAssessment\clean_points\fuzzy_TA\summary_tables\DC_fuzzy3x3_tables.xlsx", sheet_name="all_data")
    .query("type != 'change'")
    .filter(items=['uid', 'Map_static', 'GT_static', 'GT_st_3x3'])
)
df2.loc[:, 'st_correct'] = False
df2.loc[df2['Map_static'] == df2['GT_static'], "st_correct"] = True

# merge fuzzy accuracy to table
df = df.merge(df2[['uid', 'GT_st_3x3', 'st_correct']], on='uid')
df.loc[:, 'T'] = 0
df.loc[df['st_correct'], 'T'] = 1
df.loc[:, 'F'] = 0
df.loc[~df['st_correct'], 'F'] = 1
df.loc[:, 'T_3x3'] = 0
df.loc[df['GT_st_3x3'], 'T_3x3'] = 1
df.loc[:, 'F_3x3'] = 0
df.loc[~df['GT_st_3x3'], 'F_3x3'] = 1

# write data
out_path = r"C:\Users\smcdonald\Documents\Data\LULCC_2024ed\AccuracyAssessment\clean_points\distance\distance_calc.xlsx"

# assign bins
dst_fields = ['dist_to_change']
for dist_fld in dst_fields:
    mx = int(np.amax(df[dist_fld])+1)
    mn = int(np.amin(df[dist_fld])+1)
    for i in range(mn, mx+1):
        df.loc[(df[dist_fld]<=i) & (df[dist_fld]>i-1), 'bin'] = i


    df1 = df[['bin', 'T', 'F', 'T_3x3', 'F_3x3']].groupby(['bin']).sum().reset_index()

    book = load_workbook(out_path)
    writer = pd.ExcelWriter(out_path, engine = 'openpyxl')
    writer.book = book
    df1.to_excel(writer, sheet_name=dist_fld, index=False)
    writer.close()